# hisea use case workflow notebook

This notebook provides an example of running the hisea workflow.

## prerequisites
1. build the docker containers to download the data following instructions on <https://github.com/c-scale-community/use-case-hisea/tree/main/scripts/download>
2. build the docker container to preprocess the data following instructions on ... TODO

## download ERA5 data
Notes:
* path to my .cdsapirc file\
`/Users/backeb/.cdsapirc`
* path to where I am downloading the data\
`~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718`

* Other input parameters:\
`--longitude_min 22.5`\
`--longitude_max 24.5`\
`--latitude_min 36.5`\
`--latitude_max 38.5`\
`--date_min '2022-04-01'`\
`--date_max '2022-04-05'`

In [ ]:
# TODO
"""
add e.g. a input.in file where a user can define a.o.
- cdsapirclocation
- datadownloadpath
- longitude_min
- longitude_max
- latitude_min
- latitude_max
- date_min
- date_max
- modeldirectory
- preprocesseddatapath
parse this info to the docker run calls below
follow https://www.baeldung.com/linux/read-specific-line-from-file and https://stackoverflow.com/questions/4749905/how-can-i-read-a-file-and-redirect-it-to-a-variable
"""

In [ ]:
!docker run -v /Users/backeb/.cdsapirc:/root/.cdsapirc -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718:/data download-input python download_era5.py --longitude_min 22.5 --longitude_max 24.5 --latitude_min 36.5 --latitude_max 38.5 --date_min '2022-04-01' --date_max '2022-04-05'

## download CMEMS data
Notes:
* path to where I am downloading the data\
`~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718`
* Other input parameters:\
`--username bbackeberg`\
`--password iaTmwJ7D`\
`--longitude_min 22.5`\
`--longitude_max 24.5`\
`--latitude_min 36.5`\
`--latitude_max 38.5`\
`--date_min '2022-04-01'`\
`--date_max '2022-04-05'`

In [ ]:
!docker run -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718:/data download-input python download_cmems_physics.py --username bbackeberg --password iaTmwJ7D --longitude_min 22.5 --longitude_max 24.5 --latitude_min 36.5 --latitude_max 38.5 --date_min '2022-04-01' --date_max '2022-04-05'

In [ ]:
!docker run -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718:/data download-input python download_cmems_biogeochemistry.py --username bbackeberg --password iaTmwJ7D --longitude_min 22.5 --longitude_max 24.5 --latitude_min 36.5 --latitude_max 38.5 --date_min '2022-04-01' --date_max '2022-04-05'

In [ ]:
ls -lhtr ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718

## preprocessing with coastserv

Notes
* to build the preprocessing container I did the following
    * in `use-case-hisea/scripts/preprocessing/tide_physical_chemical/` do:\
    `docker build --tag preprocessing .`
* to run the preprocessing in the docker container for `boundary.py` do:\
`docker run -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718:/data/input -v /Users/backeb/Documents/REPOS/use-case-hisea/fm_model:/data/model -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718/preprocess_output:/data/output preprocessing boundary.py --interp true --simultaneous true --steric true --input /data/input --model /data/model --output /data/output`\
the above has the following input parameters:
    * the directory from which to get the downloaded CMEMS data:\
    `~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718`
    * the directory where all the model files are:\
    `/Users/backeb/Documents/REPOS/use-case-hisea/fm_model`
    * the directory where to dump the data after the preprocessing step:\
   `~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718/preprocess_output`\

In [ ]:
!docker run -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718:/data/input -v /Users/backeb/Documents/REPOS/use-case-hisea/fm_model:/data/model -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718/preprocess_output:/data/output preprocessing boundary.py --interp true --simultaneous true --steric true --input /data/input --model /data/model --output /data/output

## preprocess with hydrolib
I need to create the following files:
* salinity_east2.bc
* salinity_south2.bc
* steric_east2.bc
* steric_south2.bc
* temperature_east2.bc
* temperature_south2.bc
* tide_east2.bc
* tide_south2.bc
* uxuy_east2.bc

and I have the following files:
* DFMWAQ_east2_tmp.ext
* DFMWAQ_south2_tmp.ext
* boundary1_waq.ext
* east2.ext
* east2.pli
* south2.ext
* south2.pli
* wind_heat_waq.ext

## write .bc with hydrolib-core

In [ ]:
import xarray as xr
# xarray dependencies:
#   conda install xarray
#   conda install netcdf4
#   conda install -c conda-forge h5netcdf
#   conda install scipy
from hydrolib.core.io.bc.models import ForcingModel, TimeSeries
# to install hydrolib do 
#   `pip install hydrolib-core`
#   `conda create -n hydrolib python=3.8 -c conda-forge` and switch to hydrolib env in vsc
# additional dependency: `conda install -n hydrolib ipykernel --update-deps --force-reinstall`


pname = '/Users/backeb/OneDrive - Stichting Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718/'
ds = xr.open_dataset(pname+'cmems_thetao_bottomT_so_zos_uo_vo.nc')

import numpy
numpy.shape(ds.zos.data)

#model = ForcingModel()
#steric = TimeSeries(ds.zos.data)




In [ ]:
ds
len(numpy.shape(ds.vo))